<a href="https://colab.research.google.com/github/Farii01/Internship_PINN_work/blob/main/Senstivity_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://gricad-gitlab.univ-grenoble-alpes.fr/claveaur/pygeotools.git

In [ ]:
import pygeotools
import torch
import torch.nn as nn
import numpy

### Retrieving the data

In [ ]:
model_path = "/content/COVOBS-x2_400reals.hdf5"

pygeo = pygeotools.pygeotools()
model_name=  "COVOBS-x2_400reals.hdf5"
pygeo.loadModel(
    modelName="COVOBS-x2_400reals.hdf5",
    modelType="covobs_hdf5",
    modelPath=model_path
)

pygeo.isLoaded("COVOBS-x2_400reals.hdf5")  # Should return True

pygeotools was initialized with `verbose=True`.


True

In [ ]:
# Setting the grid
pygeo.setGrid("1deg")

# Creating the context
context = {
    "lmax": 13,
    "r": pygeo.constants["rCore"]
}

In [ ]:
# Computing the MF and SV
MF = pygeo.addMeasure("COVOBS-x2_400reals.hdf5", "MF", context)
SV = pygeo.addMeasure("COVOBS-x2_400reals.hdf5", "SV", context)

13


In [ ]:
# Retrieving the grid
_, (thetas, phis) = pygeo.getCurrentGrid()

In [ ]:
# Selecting the data
Br = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", MF, options={"component": "r", "time": 2020})
dBrdt = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", SV, options={"component": "r", "time": 2020})


### Defining the NN model

In [ ]:
node_inputs = 2
node_outputs = 2
node_layer = 64
hidden_layers = 3

In [ ]:
#  Defining the NN
# For now, it has one hidden layer with 32 nodes
# The activation functions are TANH
class CoreFlowPINN(nn.Module):
    def __init__(self):
        super(CoreFlowPINN, self).__init__()

        layers = []

        layers.append(nn.Linear(node_inputs, node_layer))
        layers.append(nn.Tanh())

        for _ in range(hidden_layers):
            layers.append(nn.Linear(node_layer, node_layer))
            layers.append(nn.Tanh())

        layers.append(nn.Linear(node_layer, node_outputs))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
r = torch.tensor(pygeo.constants["rCore"]) # placing ourselves at the CMB

def compute_loss(model, inputs, thetas_nn, phis_nn, Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn, r, λ=10):
    # Retrieving the predicted flow
    u_pred = model(inputs)

    # Retrieving the toroidal and poloidal components
    T = u_pred[:, 0:1]
    S = u_pred[:, 1:2]

    # First derivatives of T and S
    dT_dth = torch.autograd.grad(T, thetas_nn, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True)[0]
    dT_dph = torch.autograd.grad(T, phis_nn, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True)[0]
    dS_dth = torch.autograd.grad(S, thetas_nn, grad_outputs=torch.ones_like(S), create_graph=True, retain_graph=True)[0]
    dS_dph = torch.autograd.grad(S, phis_nn, grad_outputs=torch.ones_like(S), create_graph=True, retain_graph=True)[0]

    sin_th = torch.sin(thetas_nn)
    cos_th = torch.cos(thetas_nn)

    # Defining u_th and u_ph with T and S
    u_th = -dT_dph / sin_th + dS_dth
    u_ph = dT_dth + dS_dph / sin_th

    # Computing ∇h • Uh
    u_th_sin_th = u_th * sin_th
    d_u_th_sin_th_dth = torch.autograd.grad(u_th_sin_th, thetas_nn, grad_outputs=torch.ones_like(u_th_sin_th), create_graph=True, retain_graph=True)[0]
    d_u_ph_dph = torch.autograd.grad(u_ph, phis_nn, grad_outputs=torch.ones_like(u_ph), create_graph=True, retain_graph=True)[0]
    divH_uH = (1 / (r * sin_th)) * (d_u_th_sin_th_dth + d_u_ph_dph)

    # Computing ∇h Br
    gradH_Br_th = (1 / r) * dBrdth_nn
    gradH_Br_ph = (1 / (r * sin_th)) * dBrdph_nn

    # Wrapping the induction equation
    L1 = dBrdt_nn + Br_nn * divH_uH + u_th * gradH_Br_th + u_ph * gradH_Br_ph
    L1_loss = (L1**2).mean()

    # Computing L2
    L2 = divH_uH * cos_th - u_th * sin_th / r
    L2_loss = (L2**2).mean()

    # Final loss
    Loss = L1_loss + λ * L2_loss

    return Loss, L1_loss, L2_loss




In [ ]:
# For the loss history
loss_history = []

# Define patch coordinates and lambda values to explore
patches = [
    (20, 40, 20, 60),   # Original
    (15, 45, 15, 65),   # Larger
    (25, 45, 30, 70),   # Shifted
]
λ_base = 10
lambdas = [λ_base / 3, λ_base, λ_base * 3, 10**3, 10**9]  # λ/3, λ, λ×3, 10³, large λ

# Grid from pygeo
_, (thetas, phis) = pygeo.getCurrentGrid()

# Select full Br and dBrdt
Br_full = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", MF, options={"component": "r", "time": 2020})
dBrdt_full = pygeo.selectFromMeasure("COVOBS-x2_400reals.hdf5", SV, options={"component": "r", "time": 2020})

# For collecting results
results_summary = []

# Outer loop over patches
for (i1, i2, j1, j2) in patches:

    # Slice thetas/phis and fields
    thetas_bis = thetas[i1:i2]
    phis_bis = phis[j1:j2]

    Br_patch = Br_full[i1:i2, j1:j2, ...]
    dBrdt_patch = dBrdt_full[i1:i2, j1:j2, ...]
    dBrdth_patch = numpy.gradient(Br_patch, thetas_bis, axis=0)
    dBrdph_patch = numpy.gradient(Br_patch, phis_bis, axis=1)

    # Grid for this patch
    thetas_grid, phis_grid = numpy.meshgrid(thetas_bis, phis_bis, indexing="ij")
    thetas_flatten = thetas_grid.flatten()
    phis_flatten = phis_grid.flatten()

    thetas_nn = torch.tensor(thetas_flatten[:, None], dtype=torch.float32, requires_grad=True)
    phis_nn = torch.tensor(phis_flatten[:, None], dtype=torch.float32, requires_grad=True)

    Br_nn = torch.tensor(Br_patch.flatten()[:, None], dtype=torch.float32)
    dBrdt_nn = torch.tensor(dBrdt_patch.flatten()[:, None], dtype=torch.float32)
    dBrdth_nn = torch.tensor(dBrdth_patch.flatten()[:, None], dtype=torch.float32)
    dBrdph_nn = torch.tensor(dBrdph_patch.flatten()[:, None], dtype=torch.float32)

    inputs = torch.cat([thetas_nn, phis_nn], dim=1)

    # Inner loop over lambdas
    for λ in lambdas:
        model = CoreFlowPINN()
        optimizer = torch.optim.Adam(model.parameters(), 0.001)

        for epoch in range(100):
            optimizer.zero_grad()
            Loss, L1_loss, L2_loss = compute_loss(
                model, inputs, thetas_nn, phis_nn,
                Br_nn, dBrdt_nn, dBrdth_nn, dBrdph_nn,
                r, λ
            )
            Loss.backward()
            optimizer.step()

        # After training, compute flow
        model.eval()
        u_pred = model(inputs)
        T = u_pred[:, 0:1]
        S = u_pred[:, 1:2]

        dT_dth = torch.autograd.grad(T, thetas_nn, grad_outputs=torch.ones_like(T), create_graph=True)[0]
        dT_dph = torch.autograd.grad(T, phis_nn, grad_outputs=torch.ones_like(T), create_graph=True)[0]
        dS_dth = torch.autograd.grad(S, thetas_nn, grad_outputs=torch.ones_like(S), create_graph=True)[0]
        dS_dph = torch.autograd.grad(S, phis_nn, grad_outputs=torch.ones_like(S), create_graph=True)[0]

        sin_th = torch.sin(thetas_nn)

        u_theta = -dT_dph / sin_th + dS_dth
        u_phi = dT_dth + dS_dph / sin_th

        # Store result
        results_summary.append({
            "patch": (i1, i2, j1, j2),
            "lambda": λ,
            "loss": Loss.item(),
            "loss_L1": L1_loss.item(),
            "loss_L2": L2_loss.item(),
            "u_theta": u_theta.detach(),
            "u_phi": u_phi.detach()
        })

print("Done")




In [ ]:
from collections import defaultdict

def print_patchwise_loss_and_flow_differences(results_summary):
    by_patch = defaultdict(list)
    for entry in results_summary:
        by_patch[entry["patch"]].append(entry)

    print("\n=== Summary of Losses and Flow Differences ===")
    for patch, entries in by_patch.items():
        print(f"\nPatch: {patch}")
        sorted_entries = sorted(entries, key=lambda x: x["lambda"])
        for e in sorted_entries:
            print(f"  λ = {e['lambda']:10.1e} | Loss = {e['loss']:.6f} | L1 = {e['loss_L1']:.6f} | L2 = {e['loss_L2']:.6f}")

        # Compute flow diffs between every pair
        for i in range(len(sorted_entries)):
            for j in range(i + 1, len(sorted_entries)):
                e1 = sorted_entries[i]
                e2 = sorted_entries[j]

                u1_th, u1_ph = e1["u_theta"], e1["u_phi"]
                u2_th, u2_ph = e2["u_theta"], e2["u_phi"]

                delta = torch.norm(u1_th - u2_th) + torch.norm(u1_ph - u2_ph)
                ref = torch.norm(u1_th) + torch.norm(u1_ph)
                percent = (delta / ref).item() * 100

                if percent < 5:
                    status = " Stable"
                elif percent < 15:
                    status = " Moderate"
                else:
                    status = " Unstable"

                print(f"    → λ={e1['lambda']:.1e} vs λ={e2['lambda']:.1e} → Change: {percent:.2f}% → {status}")

print_patchwise_loss_and_flow_differences(results_summary)




=== Summary of Losses and Flow Differences ===

Patch: (20, 40, 20, 60)
  λ =    3.3e+00 | Loss = 2139899.750000 | L1 = 2139899.750000 | L2 = 0.000065
  λ =    1.0e+01 | Loss = 1999347.000000 | L1 = 1999347.000000 | L2 = 0.000049
  λ =    3.0e+01 | Loss = 2072402.375000 | L1 = 2072402.375000 | L2 = 0.000056
  λ =    1.0e+03 | Loss = 2061399.375000 | L1 = 2061399.375000 | L2 = 0.000055
  λ =    1.0e+09 | Loss = 2040402.750000 | L1 = 1996747.000000 | L2 = 0.000044
    → λ=3.3e+00 vs λ=1.0e+01 → Change: 17.33% →  Unstable
    → λ=3.3e+00 vs λ=3.0e+01 → Change: 14.21% →  Moderate
    → λ=3.3e+00 vs λ=1.0e+03 → Change: 9.05% →  Moderate
    → λ=3.3e+00 vs λ=1.0e+09 → Change: 15.39% →  Unstable
    → λ=1.0e+01 vs λ=3.0e+01 → Change: 6.19% →  Moderate
    → λ=1.0e+01 vs λ=1.0e+03 → Change: 9.43% →  Moderate
    → λ=1.0e+01 vs λ=1.0e+09 → Change: 3.02% →  Stable
    → λ=3.0e+01 vs λ=1.0e+03 → Change: 6.42% →  Moderate
    → λ=3.0e+01 vs λ=1.0e+09 → Change: 5.28% →  Moderate
    → λ=1.0e+03 vs

### Learning

In [ ]:
import matplotlib.pyplot as plt
plt.loglog(loss_history)

### Predictions

In [ ]:
u_th_map = u_th.reshape(Br.shape).detach().numpy()
u_ph_map = u_ph.reshape(Br.shape).detach().numpy()

In [ ]:
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs

u_th_full = numpy.zeros((thetas.size, phis.size))
u_th_full[i1:i2,j1:j2] = u_th_map

thetas_bis_deg = numpy.rad2deg(thetas)
phis_bis_deg = numpy.rad2deg(phis)

latitudes = pygeo.convertThetasToLatitudes(thetas)
longitudes = pygeo.convertPhisToLongitudes(phis)

lat_grid, lon_grid = numpy.meshgrid(latitudes, longitudes, indexing="ij")

fig = plt.figure(figsize=(15,5))

# Set the projection to Hammer and add the axes
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Aitoff())

u_max = numpy.max(numpy.abs(u_th_full))

# Use `pcolormesh` to project the data onto the map
pcol = ax.pcolormesh(lon_grid, lat_grid, u_th_full, transform=ccrs.PlateCarree(), cmap=cmocean.cm.balance, vmin=-u_max, vmax=u_max)

# Add coastlines for context
ax.coastlines()

plt.colorbar(pcol)